# Gist  

### 1. Problem
Below I am attempting a differentiable programming solution to what is arguably the oldest and most challenging inverse problem.   
The governing equation of the inverse problem we are interested in is none other than the (inhomogeneous) Hemlholtz Equation:  

$$\begin{equation}
\label{eq:helm}
\nabla^2\Psi(\mathbf{r}) + k^2 \Psi(\mathbf{r}) = -i \sigma V(\mathbf{r})\Psi(\mathbf{r})
\end{equation}
$$
where $\Psi$ is the wavefield, $V$ is the scattering potential, $k^2$ is the wavevector, and $\sigma$ is an interaction strength (or coupling constant).    
The equation above is generally applicable to all forms of radiation (lasers, x-rays...) as well as matter waves (high-energy electron beams, neutrons, etc...).  
In its most general formulation, $\Psi$ is a vector-valued complex field (i.e. to encode polarization states) or in the case of matter waves $\Psi$ is a 4-d spinor. Here, we will ignore polarization states, etc... s.t. we are dealing with scalar fields:  
$$\Psi: \mathbb{R}^3 \rightarrow \mathbb{C}, \quad V: \mathbb{R}^3 \rightarrow \mathbb{R}$$ 

The inverse problem we want to solve is given some data $D = ||\Psi_\textrm{out}||^2$, we would like to recover $V$ (and $\Psi_\textrm{in}$, most researchers only care about $V$ 🤪).  
Multiple variants of this problem arise in many fields and contexts. For instance, in imaging and sensing this inverse problem is known as object reconstruction (i.e. $V$) and in (diffraction) physics/optics it is known as the phase problem ($\arg(\Psi_\textrm{out})$ is not a measurable quantity). 

### 2. Digression #####
-----
```There is tons of approaches to solve this inverse problem (e.g. holography, tomography,) and some like Gabor and Zernike were awarded Nobel Prizes for their work on this problem ... afterall it is the **original inverse problem!**  
So, why bother finding yet another solution to this problem? well... (spoiler alert!!!🚒) none of the above solutions *actually work* in a general sense!  
And by "work in a general sense" I mean: 
1. Algo: here's the associated algorithm 
2. Data: feed it your data 
3. Solution: out comes out get a pretty good solution. 🎊

In fact, researchers, engineers, and companies have spent decades building instruments (CT scanners, microscopes, lightsources, etc...) to make the reconstruction of V easier (or in some cases, like a microscope, trivial!).   

```
-----

### 3. Paraxial Approximation and the Multi-Slice Algorithm
In what follows we use the paraxial approximation to the inhomogeneous Helmholtz equation:

$$\begin{equation}
\frac{\partial \Psi(\mathbf{r}_\perp)}{\partial z} = \frac{i \lambda}{4 \pi} \nabla_{\perp}^2 \Psi(\mathbf{r}_\perp) + i \sigma V(\mathbf{r})\Psi(\mathbf{r}_\perp)
\end{equation}
$$


In [33]:
using Plots, LinearAlgebra, Flux, NPZ, CuArrays, CUDAnative
using Flux: Optimise
using Zygote
using FFTW

FFTW.set_num_threads(Threads.nthreads())

# MSA imports
include("MSA.jl")
using .MSA

gr(html_output_format=:png)

CUDAnative.device!(3)
CuArrays.allowscalar(true)

### Utils


In [34]:
mutable struct ForwardModel
    psi::AbstractArray
    potential::AbstractArray
    simParams::Any
    name::String
end

In [35]:
function fft_shift(x)
    Zygote.nograd() do
        shifted = fftshift(x)
    end
    return shifted
end

function shift_probes(psi_k, k_x, k_y, probe_positions)
    psi_empty = CuArray{ComplexF32}(undef, (size(psi_k)[1], size(psi_k)[2], length(probe_positions)))
    psi_buff = Zygote.Buffer(psi_empty)
    psi_buff[:,:,:] = psi_empty[:,:,:]
    for (idx, pos) in enumerate(probe_positions)
        x, y = pos
        kr = k_x .* x + k_y .* y
        phase_shift = exp.(2.f0 * pi * im .* kr)
        psi_buff[:,:,idx] = ifft(psi_k .* phase_shift, [1,2])
        psi_buff[:,:,idx] = fftshift(copy(psi_buff[:,:,idx]))
        norm = sqrt(sum(abs2, copy(psi_buff[:,:,idx])))
        psi_buff[:,:,idx] = copy(psi_buff[:,:,idx]) ./ norm
    end
    psi = copy(psi_buff)
    return psi
end

function shift_probes(psi_k, k_x, k_y, x, y)
    psi_buff = Zygote.Buffer(psi_k)
    kr = k_x .* x + k_y .* y
    phase_shift = exp.(2.f0 * pi * im .* kr)
    psi_buff[:,:,:] = ifft(psi_k .* phase_shift, [1,2])
    for idx in 1:length(x)
        psi_buff[:,:,idx] = fftshift(copy(psi_buff[:,:,idx]))
    end
    norm = sqrt.(sum(abs2.(copy(psi_buff)), dims=(1,2)))
    norm_vec = reshape(norm, 1, 1, :)
    psi_buff = copy(psi_buff) ./ norm
    return copy(psi_buff)
end

function forward(model::ForwardModel, args)
    amp, k_x, k_y, k_arr, x, y = args
    psi_k = amp .* exp.(-1.0f0 * im .* model.psi)
    psi_k_stack = repeat(psi_k, outer=(1,1,length(x)))
    probes = shift_probes(psi_k_stack, k_x, k_y, x, y)
    psi_out = MSA.multislice(probes, model.potential, k_arr, model.simParams)
    psi2_out = abs.(psi_out)
    return psi2_out
end

function loss(model::ForwardModel, psi2_trg, psik_trg, args; verbose=false)
    psi2_pred = forward(model, args)
    mse = norm(psi2_pred  .- psi2_trg, 2)
#     v_constr = norm(imag.(model.potential), 2) 
#     psi_constr = norm(imag.(model.psi), 2) 
#     if verbose
#         Zygote.nograd(println("Data loss: $(round(mse;digits=4)), V_constr: $(round(v_constr;digits=4)), Psi_constr: $(round(psi_constr; digits=4))"))
#     end
    return mse #+ v_constr + psi_constr
end

function sgd_update!(model::ForwardModel, grads, η = 0.01; verbose=false)
    if verbose 
        @info("Gradients:
            ∂V=$(round.(grads.potential; digits=2))
            ∂ѱ=$(round.(grads.psi; digits=2))")
        @info("before sgd update:
        V'=$(round.(model.potential; digits=2))
        ѱ'=$(round.(model.psi; digits=2))")
    end
    model.psi .-= η .* grads.psi
    model.potential .-= η .* grads.potential
    
    if verbose
        @info("after sgd update:
        V'=$(round.(model.potential; digits=2))
        ѱ'=$(round.(model.psi; digits=2))")
    end
end

function comp_plot(psi_out, psi_in, V)
    l = @layout [a b c]
    p1 = heatmap(abs2.(psi_in[:,:,1]) .^ 0.25, aspect_ratio=1, framestyle = :none)
    p2 = heatmap(psi_out[:,:,1] .^ 0.1, aspect_ratio=1, framestyle = :none)
    p3 = heatmap(sum(V, dims=3)[:,:,1], aspect_ratio=1, framestyle = :none, color = :vibrant_grad)
    heatmap(p1, p2, p3, layout=l, title=["Psi_in" "Data" "V"], size=(1200,400))
    
end

function V_plot(V_true, V_pred)
    l = @layout [a b]
    p1 = heatmap(sum(real.(V_true), dims=3)[:,:,1], aspect_ratio=1, framestyle = :none)
    p2 = heatmap(sum(real.(V_pred), dims=3)[:,:,1], aspect_ratio=1, framestyle = :none)
    heatmap(p1, p2, layout=l , title=["V-target" "V-predicted"], size=(800,400))
end
    
function grad_plot(grads)
    l = @layout [a b]
    p1 = heatmap(angle.(grads.psi[:,:,1]), aspect_ratio=1, framestyle = :none)
    p2 = heatmap(angle.(grads.potential)[:,:,1], aspect_ratio=1, framestyle = :none)
    heatmap(p1, p2, layout=l, title=["arg(Grad_Psi)" "arg(Grad_V)"], size=(800,400))
end
    
function diff_plot(psi_out_pred, psi_out_true;clims=:auto)
    pred = sum(psi_out_pred;dims=3)[:,:,1]
    gtruth = sum(psi_out_true;dims=3)[:,:,1]
    heatmap((pred .- gtruth) ./ gtruth .* 100, 
        aspect_ratio=1, framestyle= :none, title="% Difference", clims= clims)
end

function build_psi(amp, psi_k_phase)
    psi_k = amp .* exp.(-1.0f0 * im .* psi_k_phase)
    psi_x = ifft(psi_k, [1,2])
    psi_x .= fftshift(psi_x, [1,2])
    norms = sqrt.(sum(abs2, psi_x, dims=[1,2]))
    psi_x ./= norms
    return psi_x
end
    
    
    

build_psi (generic function with 1 method)

#### Set Simulation Params

In [36]:
soft_aperture = Dict("type" => "soft", 
                     "factor" => 25.0)
hard_aperture = Dict("type" => "hard")
probe = Dict("phase_error" => "spherical", 
             "C1" => 1, 
             "C3" => 1, 
             "C5" => 1, 
             "Scherzer" => false)
sigma = 0.01
num_slices = 1
slice_thickness = 0.25
bandlimit = 1.0
simParams = MSA.SimulationState(0.05, 128, 4, 0.02, soft_aperture, probe, sigma, slice_thickness, bandlimit)
# probe_positions = MSA.get_probe_coordinates(simParams; grid_steps=[12,12], fraction=0.6, origin=[0., 0.]);

In [37]:
function get_probe_coordinates(simParams::MSA.SimulationState; fraction=0.5, origin=[0,0], grid_steps=[8,8])
    grid_range_start = (0.5 .+ origin * simParams.sampling .- simParams.sampling * fraction/4) ./ 2
    grid_range_stop = (0.5 .+ origin * simParams.sampling .+ simParams.sampling * fraction/4) ./ 2
    x_range = range(grid_range_start[1], stop=grid_range_stop[1], length=grid_steps[1])
    y_range = range(grid_range_start[2], stop=grid_range_stop[2], length=grid_steps[2])
    probe_positions = [[-i, j] for i in x_range, j in y_range]
    probe_positions = hcat(probe_positions...)
    x, y = probe_positions[1,:], probe_positions[2,:]
    x = reshape(x, 1, 1, :)
    y = reshape(y, 1, 1, :)
    return (x, y)
end

get_probe_coordinates (generic function with 5 methods)

#### Load V

In [38]:
# load a potential
v_file = npzread("data/potential_445.npy")
v_guess = npzread("data/potential_5419.npy")
if simParams.sampling == 128
    crop_top = map(Int, [size(v_file)[1] * (1/4), size(v_file)[2] * (3/4)])
elseif simParams.sampling == 64
    crop_top = map(Int, [size(v_file)[1] * (1/8), size(v_file)[2] * (3/8)])
else
    crop_top = (1, size(v_file)[2]+1)
end
slice = range(crop_top[1], stop=crop_top[2]-1)
v_file = v_file[slice, slice]
v_guess = v_guess[slice,slice]

V_trg = randn(ComplexF32, (simParams.sampling, simParams.sampling, num_slices)) 
for idx in 1:size(V_trg, 3)
    V_trg[:,:,idx] = im * v_file
end

In [46]:
simParams_cu = cu(simParams)

Main.MSA.SimulationState(0.05f0, 128, 4.0f0, 0.02f0, Dict{String,Any}("factor" => 25.0,"type" => "soft"), Dict{String,Any}("C1" => 1,"phase_error" => "spherical","C3" => 1,"Scherzer" => false,"C5" => 1), 0.01f0, 0.25f0, 1.0f0)

In [43]:
psi_k, k_rad = MSA.build_probe(cu(simParams));

In [45]:
typeof(k_rad)

Complex{Float64}

### Generate Solution by simulating $F$

In [40]:
k_size = (simParams.sampling,simParams.sampling)
@info("Initiate Wavefunction and Scattering Potential...") 
psi, psi_k, k_arr = MSA.build_probe(simParams, cu(probe_positions))
psi_trg = psi
cpy_trg = copy(psi)
V_trg = randn(Float32, (k_size..., num_slices)) 
for idx in 1:size(V_trg, 3)
    V_trg[:,:,idx] = v_file
end
MSA.multislice!(psi_trg, cu(V_trg), cu(k_arr), simParams)
psi2_trg = abs.(psi_trg)
comp_plot(Array(psi2_trg[:,:,5]), Array(angle.(psi_k)), V_trg)

┌ Info: Initiate Wavefunction and Scattering Potential...
└ @ Main In[40]:2


UndefVarError: UndefVarError: probe_positions not defined

### Initiate Differentiable Forward Model, $\partial F$

In [131]:
@info("Initiate Forward Model...")
x, y = get_probe_coordinates(simParams; grid_steps=[12,12], fraction=0.6, origin=[0., 0.])
x, y = map(cu, (x, y))
_, psi_k, k_arr = MSA.build_probe(simParams)
k_x, k_y = MSA.get_kspace_coordinates(simParams)
psi_mixing = 0.9
V_mixing = 0.9
psi_k_phase = angle.(psi_k) 
psi_in_phase = psi_mixing * psi_k_phase + (1 - psi_mixing) * randn(Float32, size(psi_k))
psi_in_amp = abs.(psi_k) 
V_in = V_mixing * V_trg + (1 - V_mixing) * randn(Float64, size(V_trg))
psi_k = repeat(cu(psi_k), outer=(1,1,length(x)))
k_x = repeat(cu(k_x), outer=(1,1,length(x)))
k_y = repeat(cu(k_y), outer=(1,1, length(x)))
V_in = V_in .+ 0.0f0 * im
psi_in_phase = psi_in_phase .+ 0.0f0 * im 
args = (cu(psi_in_amp), k_x, k_y, cu(k_arr), x, y)
model = ForwardModel(cu(psi_in_phase), cu(V_in), cu(simParams), "∂F")
res = Array(sum(forward(model, args); dims=3))
comp_plot(res, real.(psi_in_phase), real.(V_in))

┌ Info: Initiate Forward Model...
└ @ Main In[131]:1


### Differentiate $\partial F$

In [132]:
# model gradients
@info("Differentiating Forward Model...")
grads = gradient(model) do m
    return loss(m, psi2_trg, psi_k, args)
end
grads = grads[1][]
# grad_plot(grads)

┌ Info: Differentiating Forward Model...
└ @ Main In[132]:2


MethodError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:600
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:600
  iterate(!Matched::ExponentialBackOff) at error.jl:218
  ...

### Difference between initial $\partial F$ and solution

In [53]:
diff_plot(abs2.(forward(model, args)), psi2_trg, clims=(0,100))

CUDAdrv.CuError: CUDA error: an illegal memory access was encountered (code 700, ERROR_ILLEGAL_ADDRESS)

### Optimize

In [ ]:
ilr = 0.1
opt = ADAM(ilr, (0.8, 0.99))
@info("Running train loop")
idx = 0
loss_val = loss(model, psi2_trg, psi_k, args)
max_iter = 2e2
num_logs = Int(min(max_iter/4, 250))
verbose = false
model_hist = Array{Tuple}(undef,num_logs)
loss_hist = Array{Float32}(undef, num_logs)
iter_hist = Array{Int}(undef, num_logs)
hist_idx = 1
@time while idx < max_iter && loss_val > 1e-4
    lr =  ilr * (idx + 1) .^ (-0.2)
#     lr = ilr
    opt.eta = lr
    if mod(idx, max_iter ÷ num_logs) == 0
        loss_val = loss(model, psi2_trg, psi_k, args; verbose=true)
        println("lr=$(round(lr;digits=4)), Iteration=$idx, Loss=$(round(loss_val;digits=4))")
        model_hist[hist_idx] = (model.psi[:,:,1], model.potential[:,:,:])
        loss_hist[hist_idx] = loss_val
        iter_hist[hist_idx] = idx
        global hist_idx += 1
    end
    grads= Zygote.gradient(model) do m
        return loss(m, psi2_trg, psi_k, args)
    end
    grads = grads[1][]
#     sgd_update!(model, grads, 1e-3)
    Optimise.update!(opt, model.psi, grads.psi)
    Optimise.update!(opt, model.potential, grads.potential)
    global idx += 1
end

In [ ]:
plot(iter_hist[1:hist_idx-1], loss_hist[1:hist_idx-1], lw=2, title="Loss vs. # Grad updats", label="Loss")


### Difference between optimized $\partial F$ and solution

In [ ]:
diff_plot(forward(model, args), psi2_trg; clims=(0,100))

### Optimization History of $(\bar\Psi_{\textrm{in}}, V)$

In [ ]:
anim = @animate for ((psi,potential), iter) in zip(model_hist[1:hist_idx-1], iter_hist[1:hist_idx-1])
    psi = real.(psi[:,:,1]) 
#     pot = sum(abs2.(potential), dims=3)[:,:,1]
    pot = sum(real.(potential), dims=3)[:,:,1]
    l = @layout [a b]
    p1 = heatmap(psi, aspect_ratio=1, framestyle = :none, color= :vibrant_grad)
    p2 = heatmap(pot , aspect_ratio=1, framestyle = :none, color= :vibrant_grad)
    heatmap(p1, p2, layout=l, title=["Psi_in\n Iterations=$iter" "V"], size=(800,400))
    end

In [ ]:
gif(anim, "gifs/diffProg_pot_445_128x128.gif", fps=5)

In [ ]:
comp_plot(sum(forward(model, args); dims=3), model.psi, real.(model.potential))

In [ ]:
V_plot(V_trg, model.potential .- minimum(real.(model.potential)))


In [ ]:
psi_x = build_psi(psi_in_amp, model.psi)
psi_x_trg, psi_k, k_arr = MSA.build_probe(simParams)
V_plot(abs.(psi_x_trg).^ 0.25, abs.(psi_x) .^ 0.25)